## Boosting

Boosting is a sequential approach where the training of a model
learner at a given step applies to "residuals" from the model fitted
at the previous steps. This way, the current step improves on data
points where the cumulative performance is not good. Boosting produces
an ensemble model that in general less biased than the weak learners
that compose it. 

Unlike bagging, boosting cannot be done in parallel.

Boosting methods differ on how they create and aggregate the weak
learners during the sequential process.

See [Joseph Rocca's
post](https://towardsdatascience.com/ensemble-methods-bagging-boosting-and-stacking-c9214a10a205);
[Jason Brownlee's
blog](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/).


### Adaptive Boosting (AdaBoost)
Adaptive boosting updates the weights attached to each of the training
dataset observations. The ensemble model is a weighted sum of the weak
learners. Instead of trying to solve it in one single shot (finding
all the coefficients and weak learners that give the best overall
additive model), an iterative optimisation process is
much more tractable, even if it can lead to a sub-optimal solution.


For illustration, consider a binary classification problem, with $N$
observations and a given family of weak models. To initialize, all the
observations have the same weights $1/N$. Repeat over each weak model:
+ fit the best possible weak model with the current observations
  weights;
+ compute the value of the update coefficient that is some kind of
  scalar evaluation metric of the weak learner that indicates how much
  this weak learner should be taken into account into the ensemble
  model;
+ update the strong learner by adding the new weak learner multiplied
  by its update coefficient;
+ compute new observations weights that express which observations we 
  would like to focus on at the next iteration (weights of observations 
  wrongly predicted by the aggregated model increase and weights of 
  the correctly predicted observations decrease).

### Gradient Boosting

Gradient boosting updates the dataset in each step. It casts the
problem into a gradient descent one: at each iteration we fit a weak
learner to the negative gradients of the current fitting error
(pseudo-residuals) with respect to the current ensemble model. In the
regression setting, the negative gradient is the residual from the
current ensemble model.

For illustration, continue with the binary classification
problem. Initialize the pseudo-residuals as the observation outcomes.
Repeat over the following steps:
+ fit the best possible weak learner to pseudo-residuals (approximate
  the negative gradients with respect to the current strong learner);
+ compute the value of the optimal step size that defines by how much
  we update the ensemble model in the direction of the new weak
  learner;
+ update the ensemble model by adding the new weak learner multiplied
  by the step size (make a step of gradient descent);
+ compute new pseudo-residuals that indicate, for each observation, in
  which direction we would like to update next the ensemble model
  predictions.

Gradient boosting uses a gradient descent approach and can be easily
adapted to a large number of loss functions. It can be considered as a
generalization of adaboost to arbitrary differentiable loss functions.



#### Stochastic gradient boosting

Stochastic gradient boosting (SGD) is a stochastic version of gradient
boosting. At each iteration, a subsample of the training data is drawn
at random (without replacement) from the full training dataset. The
randomly selected subsample is then used, instead of the full sample,
to fit the base learner. It reduce the dependence between the trees
in the sequence in gradient boosting models.



#### Extreme gradient boosting
Extreme gradient boosting is a specific implementation of the gradient
boosting method which uses more accurate approximations to find the
best tree model. It employs a number of nifty tricks that make it
exceptionally successful, particularly with structured data. 

+ computing second-order gradients, i.e. second partial derivatives of
  the loss function (similar to Newton’s method), which provides more
  information about the direction of gradients and how to get to the
  minimum of our loss function. While regular gradient boosting uses
  the loss function of our base model (e.g. decision tree) as a proxy
  for minimizing the error of the overall model, XGBoost uses the 2nd
  order derivative as an approximation.
+ advanced regularization (L1 & L2), which improves model
  generalization.
+ parallelized.



### Boosting with NYC Crash Data Example
We can demonstrate the implementation of AdaBoost, Stochastic Gradient Boost and XG Boost in Python using sklearn.
An interesting metric to predict is whether or not the incident resulted in a casualty.

We will use the following predictors to train our model.

+ `hour`: the hour at which the crash occured
+ `borough`: which borough the crash took place in
+ `num_vehicles_involved`: number of vehicles involved in a crash
+ `weekday`: day of week of the crash, where 0 represents Monday and 6 represents Sunday

In [ ]:
!pip install sklearn


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from datetime import datetime

url = 'https://raw.githubusercontent.com/statds/ids-s22/main/notes/data/nyc_mv_collisions_202201.csv'
nyc_crash = pd.read_csv(url)
nyc_crash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7659 entries, 0 to 7658
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   CRASH DATE                     7659 non-null   object 
 1   CRASH TIME                     7659 non-null   object 
 2   BOROUGH                        5025 non-null   object 
 3   ZIP CODE                       5025 non-null   float64
 4   LATITUDE                       7097 non-null   float64
 5   LONGITUDE                      7097 non-null   float64
 6   LOCATION                       7097 non-null   object 
 7   ON STREET NAME                 5625 non-null   object 
 8   CROSS STREET NAME              3620 non-null   object 
 9   OFF STREET NAME                2034 non-null   object 
 10  NUMBER OF PERSONS INJURED      7659 non-null   int64  
 11  NUMBER OF PERSONS KILLED       7659 non-null   int64  
 12  NUMBER OF PEDESTRIANS INJURED  7659 non-null   i

In [ ]:
# Creating a casualty variable to determine how many people are killed or injured in a crash
nyc_crash['casualty'] = nyc_crash.apply(lambda row: 1 if (row['NUMBER OF PERSONS KILLED'] + row['NUMBER OF PERSONS INJURED']) > 0 else 0, axis = 1)

# Casting to category
nyc_crash['casualty'] = nyc_crash['casualty'].astype('category')

# Creating a hour variable to determine the hour at which the incident took place
def get_hour(entry):
  time = datetime.strptime(entry, "%H:%M")
  return int(time.hour)

nyc_crash['hour'] = nyc_crash['CRASH TIME'].apply(get_hour)

# Creating a number of vehicles involved variable
contributing_factors = ['CONTRIBUTING FACTOR VEHICLE 1', 'CONTRIBUTING FACTOR VEHICLE 2',
                        'CONTRIBUTING FACTOR VEHICLE 3', 'CONTRIBUTING FACTOR VEHICLE 4',
                        'CONTRIBUTING FACTOR VEHICLE 5']

nyc_crash['num_vehicles_involved'] = len(nyc_crash[contributing_factors].columns) - nyc_crash[
                                         contributing_factors].isnull().sum(axis = 1)

# Creating a day of the week variable where 0 represents Monday and 6 represents Sunday
def get_weekday(entry):
  entry = datetime.strptime(entry, "%m/%d/%Y")
  return entry.weekday()
  
nyc_crash['weekday'] = nyc_crash['CRASH DATE'].apply(get_weekday)
nyc_crash.rename(columns = {'BOROUGH': 'borough'}, inplace = True)
nyc_crash.dropna(subset=['borough'], inplace=True)
nyc_crash[['borough', 'hour', 'num_vehicles_involved', 'weekday']].isnull().sum()


borough                  0
hour                     0
num_vehicles_involved    0
weekday                  0
dtype: int64

In [ ]:
features = nyc_crash[['hour', 'num_vehicles_involved', 'weekday']] # Predictors
labels = np.array(nyc_crash['casualty']) # Outcome to predict

# In-place one-hot endcoding for borough variable
pd.get_dummies(features)

# Creating test and train sets with 20% of data going to test set
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2, random_state = 42)

Now, we will initialize an Adaptive Boosting, Gradient Boosting, Stochastic Gradient Boosting, and XGBoosting classifiers. We can compare their accuracy score, which is the number of correctly classified observations / total number of observations.

In [ ]:
ada_booster = AdaBoostClassifier(random_state=42)
ada_booster.fit(x_train, y_train)
ada_booster.score(x_test, y_test)

0.7223880597014926

In [ ]:
gradient_booster = GradientBoostingClassifier(random_state=42)
gradient_booster.fit(x_train, y_train)
gradient_booster.score(x_test, y_test)

0.7243781094527363

In [ ]:
stochastic_gradient_booster = GradientBoostingClassifier(subsample=0.5, random_state=42)
stochastic_gradient_booster.fit(x_train, y_train)
stochastic_gradient_booster.score(x_test, y_test)

0.718407960199005

In [ ]:
xg_booster = XGBClassifier(random_state=42)
xg_booster.fit(x_train, y_train)
y_pred = xg_booster.predict(x_test)
accuracy_score(y_test, y_pred)

0.7203980099502487

All of these boosting algorithms present similar accuracy scores for this dataset. However, they all use different methods of computation. Perhaps for larger data sets and with more focused hyperparameter tuning, the difference between these algorithms would become more apparent.